# REI505M Machine Learning - Final project
### Due: --------

**Names**: Axel Kristján Axelsson, Bjarni Haukur Bjarnason <br />
**Email**: aka30@hi.is, bhb23@hi.is

**1. Image classification**

s


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image

from data_loader import MyDataLoader
from nn_helper import save_model, load_model


%load_ext autoreload 
%autoreload 2

norm = (128,128)

dl = MyDataLoader("combine2", norm)

In [26]:
dl.normalize_train_data(norm_size=norm)

try:
    del X
    del y

except:
    pass

X, y = dl.get_lab_data()
dl.numpy_dump(X, "X")
dl.numpy_dump(y, "y")

100%|██████████| 102/102 [01:30<00:00,  1.12it/s]


In [15]:
# X = np.squeeze(dl.numpy_load("X"))
X = dl.numpy_load("X")
y = dl.numpy_load("y")
X_lab = X/100
y_lab = y/128

In [40]:
print(X_lab.shape)
print(y_lab.shape)
print(np.max(X_lab))
print(np.max(y_lab))
print(np.min(X_lab))
print(np.min(y_lab))

(11636, 128, 128, 1)
(11636, 128, 128, 2)
1.0
0.7578125
0.0
-0.8359375


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_lab, y_lab, test_size = 0.3, random_state = 3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.33, random_state = 3)

In [27]:
model = load_model("model_a")

canvas = np.zeros((100,128,128,3))
pred = model.predict(X_test)
canvas[:,:,:,0] = X_test[:100,:,:,0]*100
canvas[:,:,:,1:] = pred[:100,:,:]*128

canvas = color.lab2rgb(canvas)*255
canvas = canvas.astype(np.uint8)


C:\Users\bjarn\AppData\Roaming\Python\Python39\site-packages\skimage\color\colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 28 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


In [30]:
img = Image.fromarray(canvas[5])
print(np.max(img))
print(np.min(img))
img.show()

255
0


In [6]:
#keras.backend.clear_session()

1067581584


In [7]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam




from neural_nets import NeuralNets

nn = NeuralNets(norm)


# model = load_model("model_a")
model = nn.model_a()


opt = Adam(learning_rate=1e-3)

model.compile(optimizer=opt, loss='mse')

# img_shape = (128, 128, 1)
# kernel_size = (3,3)
# num_filters = 64


# model = Sequential()
# model.add(InputLayer(input_shape=img_shape))
# model.add(Conv2D(num_filters, kernel_size, activation='relu', padding='same'))
# model.add(Conv2D(num_filters, kernel_size, activation='relu', padding='same', strides=2))
# model.add(Conv2D(num_filters*2, kernel_size, activation='relu', padding='same'))
# model.add(Conv2D(num_filters*2, kernel_size, activation='relu', padding='same', strides=2))
# model.add(Conv2D(num_filters*4, kernel_size, activation='relu', padding='same'))
# model.add(Conv2D(num_filters*4, kernel_size, activation='relu', padding='same', strides=2))
# model.add(Conv2D(num_filters*8, kernel_size, activation='relu', padding='same'))
# model.add(Conv2D(num_filters*4, kernel_size, activation='relu', padding='same'))
# model.add(Conv2D(num_filters*2, kernel_size, activation='relu', padding='same'))
# model.add(UpSampling2D((2, 2)))
# model.add(Conv2D(num_filters, kernel_size, activation='relu', padding='same'))
# model.add(UpSampling2D((2, 2)))
# model.add(Conv2D(32, kernel_size, activation='relu', padding='same'))
# model.add(Conv2D(2, kernel_size, activation='sigmoid', padding='same'))

# opt = Adam(learning_rate=1e-3)
# model.compile(optimizer=opt, loss='mse')

In [8]:
num_epochs = 1
nr = 42
canvas = np.zeros((128,128,3))
for i in range(num_epochs):
    model.fit(X_train, y_train, batch_size = 64, epochs = 3)
    

save_model(model, "model_a", brave=True)

Epoch 1/3
128/128 [==============================] - 32s 200ms/step - loss: 0.0457
Epoch 2/3
128/128 [==============================] - 23s 180ms/step - loss: 0.0383
Epoch 3/3
128/128 [==============================] - 22s 176ms/step - loss: 0.0383
INFO:tensorflow:Assets written to: model_a\assets


In [18]:
nr42 = X_test[42]
print(nr42.shape)
bla = np.expand_dims(X_test[42], axis=0)



pred = model.predict(bla)
pred = pred*128
canvas[:,:,0] = bla[:,:,0]*100
canvas[:,:,1:] = pred
canvas = canvas.astype(np.uint8)
img = Image.fromarray(canvas)
img.save("mynd42.png")

(128, 128, 1)


In [11]:
nr = 42
print(X_test[nr].shape)
# canvas = np.zeros((128,128,3))
pred = model.predict(X_test)
# pred = pred*128
# canvas[:,:,0] = X_test[nr,:,:,0]*100
# canvas[:,:,1:] = pred[nr]
# canvas = canvas.astype(np.uint8)
# img = Image.fromarray(canvas)
# img.save("mynd" + i + ".png")

(128, 128, 1)


In [16]:
nr = 42
print(X_test[nr].shape)
canvas = np.zeros((128,128,3))
predNr = pred[nr]*128
canvas[:,:,0] = X_test[nr,:,:,0]*100
canvas[:,:,1:] = predNr[nr]
canvas = canvas.astype(np.uint8)
img = Image.fromarray(canvas)
img.save("mynd1.png")

(128, 128, 1)


In [ ]:
from PIL import Image

canvas = np.zeros((128,128,3))

nr = 10

canvas[:,:,0] = X_test[nr,:,:,0]*100
canvas[:,:,1:] = pred[nr]
canvas = canvas.astype(np.uint8)

img = Image.fromarray(canvas)
img.show()

In [ ]:

mynd = pred[1]
fokking = np.zeros((128,128,3))
print(X_test[1].shape)
fokking[:,:,0] = X_test[1]
fokking[:,:,1:] = mynd
fokking = fokking.astype(int)
print(np.max(fokking))
print(np.min(fokking))

In [ ]:
import numpy as np

print(X.shape)
print(y.shape)
print(np.max(X))
print(np.min(X))
print(np.max(y))
print(np.min(y))

In [ ]:
from PIL import Image

img = Image.fromarray(drasl)

imgO = Image.fromarray(X_test[1])
# imgTrue = Image.fromarray(y_test[1], 'RGB')
# img = Image.fromarray(mynd1, 'RGB')
imgO.show()
# imgTrue.show()
img.show()

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
print(X.shape)
print(X[11001])


In [ ]:


# print(X.shape)
# print(X_test.shape)
# im = Image.fromarray(X_train[0], mode="YCbCr")
# im.show()
# print(X_test[0])

In [14]:
from PIL import Image
import numpy as np
from skimage import color

img = Image.open("test.jpg")
img = img.resize((500,500))

arr1 = np.array(img)
arr2 = color.rgb2lab(img)

canvas = np.zeros((500,500,3))
canvas[:,:,0] = arr2[:,:,0]/100
canvas[:,:,1:] = arr2[:,:,1:]/128

canvas[:,:,0]  = canvas[:,:,0]*100
canvas[:,:,1:] = canvas[:,:,1:]*128

canvas = color.lab2rgb(canvas)*255
canvas = canvas.astype(np.uint8)
canvas = Image.fromarray(canvas)

arr3 = color.lab2rgb(color.rgb2lab(img))*255

img1 = Image.fromarray(arr1)
img2 = Image.fromarray(arr2.astype(np.uint8))
img3 = Image.fromarray(arr3.astype(np.uint8))

img1.save("img1.jpg")
img2.save("img2.jpg")
img3.save("img3.jpg")
canvas.save("canvas.jpg")
# print(arr1.shape)
# print(arr2.shape)
# print(arr1[0,0,0])
# print(arr2[0,0,0])
# print(type(arr1[0,0,0]))
# print(type(arr2[0,0,0]))

# arr2 = arr2.astype(np.uint8)



# print(type(arr1[0,0,0]))
# print(type(arr2[0,0,0]))

# print((arr1!=arr2).sum()/float(arr1.size))

# img1.show()
# img2.show()

In [ ]:
from PIL import Image
import numpy as np

img = Image.open("test.jpg")
img = img.resize((500,500))
img.show()